<a href="https://colab.research.google.com/github/nicoloceneda/Python-edu/blob/master/TensorFlow_Custom_Layer_and_Models_with_Keras_API.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TensorFlow - Custom Layer and Models with Keras API 
---



In [0]:
import tensorflow as tf
from tensorflow.keras import layers

## Adding weights
A Layer is the main data structure that encapsulates a state (the weights w and b) and a transformation from inputs to outputs (the forward pass defined in call). 

**Add trainable weights** to a layer `manually` or using `add_weight` in the `__init__` method

In [2]:
# Manually
class Linear(layers.Layer):

  def __init__(self, units=32, input_dim=32):
    super(Linear, self).__init__()
    w_init = tf.random_normal_initializer()
    self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units), dtype='float32'), trainable=True)
    b_init = tf.zeros_initializer()
    self.b = tf.Variable(initial_value=b_init(shape=(units, ), dtype='float32'), trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

x = tf.ones((2, 2))

linear_layer = Linear(units=4, input_dim=2)
y = linear_layer(inputs=x)
tf.print(y)

[[-0.0480966978 -0.0887475386 -0.0465916097 -0.0243805367]
 [-0.0480966978 -0.0887475386 -0.0465916097 -0.0243805367]]


In [3]:
# Using add_weight
class Linear(layers.Layer):

  def __init__(self, units=32, input_dim=32):
    super(Linear, self).__init__()
    self.w = self.add_weight(shape=(input_dim, units), initializer='random_normal', trainable=True)
    self.b = self.add_weight(shape=(units, ), initializer='zeros', trainable=True)

  def call(self, inputs):
    return tf.linalg.matmul(inputs, self.w) + self.b

  x = tf.ones((2, 2))

  linear_layer = Linear(units=4, input_dim=2)
  y = linear_layer(inputs=x)
  tf.print(y)

[[-0.117562369 0.0193092749 0.164690942 -0.109509379]
 [-0.117562369 0.0193092749 0.164690942 -0.109509379]]


**Add non-trainable weights** (which are meant not to be taken into account during the backpropagation process of training) to a Layer `manually`. 

In [4]:
class ComputeSum(layers.Layer):

  def __init__(self, input_dim=32):
    super(ComputeSum, self).__init__()
    self.total = tf.Variable(initial_value=tf.zeros((input_dim, )), trainable=False)

  def call(self, inputs):
    self.total.assign_add(tf.reduce_sum(inputs, axis=0))
    return self.total

x = tf.ones((2, 2))

my_sum = ComputeSum(input_dim=2)
y = my_sum(inputs=x)
tf.print(y)
y = my_sum(inputs=x)
tf.print(y)

[2 2]
[4 4]


**Defer weight creation** until the shape of the inputs is known using `add_weight` in the `build` method. Doing so, the weights are created dynamically the first time the layer is called. This is the best practice as in many cases the size of the inputs is not known in advance.

In [5]:
class Linear(layers.Layer):

  def __init__(self, units=32):
    super(Linear, self).__init__()
    self.units = units

  def build(self, input_shape=32):
    self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
    self.b = self.add_weight(shape=(self.units, ), initializer='zeros', trainable=True)
  
  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

x = tf.ones((2, 2))

linear_layer = Linear(units=4)
y = linear_layer(inputs=x)
tf.print(y)

[[-0.0343586169 -0.0175083317 -0.121679261 -0.0353164859]
 [-0.0343586169 -0.0175083317 -0.121679261 -0.0353164859]]


Define **layers recursively** (a layer instance is assigned as attribute of another layer) by instantiating them in the `__init__` method.

In [6]:
class Linear(layers.Layer):

  def __init__(self, units=32):
    super(Linear, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
    self.b = self.add_weight(shape=(self.units, ), initializer='zeros', trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

class MLPBlock(layers.Layer):

  def __init__(self):
    super(MLPBlock, self).__init__()
    self.linear_1 = Linear(units=32)
    self.linear_2 = Linear(units=32)
    self.linear_3 = Linear(units=1)

  def call(self, inputs):
    x = self.linear_1(inputs)
    x = tf.nn.relu(x)
    x = self.linear_2(x)
    x = tf.nn.relu(x)
    return self.linear_3(x)

x = tf.ones(shape=(3, 64))

mlp = MLPBlock()
y = mlp(inputs=x)
tf.print(y)

[[-0.0070726471]
 [-0.0070726471]
 [-0.0070726471]]


## Adding losses

**Add a loss** tensor using `add_loss` in the `call` method

In [0]:
class ActivityRegularizationLayer(layers.Layer):

  def __init__(self, rate=1e-2):
    super(ActivityRegularizationLayer, self).__init__()
    self.rate = rate 

  def call(self, inputs):
    self.add_loss(self.rate * tf.math.reduce_sum(inputs))
    return inputs

**Retrieve the loss** using `layer.losses` (this property is reset at the start of every call to the top-level layer, so that it always contains the loss created during the last forward pass). 

In [0]:
class ActivityRegularizationLayer(layers.Layer):

  def __init__(self, rate=1e-2):
    super(ActivityRegularizationLayer, self).__init__()
    self.rate = rate 

  def call(self, inputs):
    self.add_loss(self.rate * tf.math.reduce_sum(inputs))
    return inputs

class OuterLayer(layers.Layer):

  def __init__(self):
    super(OuterLayer, self).__init__()
    self.activity_reg = ActivityRegularizationLayer(rate=1e-2)

  def call(self, inputs):
    return self.activity_reg(inputs)

x = tf.zeros(1, 1)

layer = OuterLayer()
assert len(layer.losses) == 0  # No losses yet since the layer has never been called
_ = layer(x)
assert len(layer.losses) == 1  # We created one loss value
_ = layer(x)
assert len(layer.losses) == 1  # This is the loss created during the call above

tf.Tensor([0.], shape=(1,), dtype=float32)
tf.Tensor([0.], shape=(1,), dtype=float32)


## Serializing layers
**Serialize layers** (recreate a layer from its configuration, as part of a functional model) using `get_config`

In [0]:
class Linear(layers.Layer):

  def __init__(self, units=32):
    super(Linear, self).__init__()
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
    self.b = self.add_weight(shape=(self.units, ), initializer='zeros', traibale=True)

  def call(self, inputs):
    return tf.linalg.matmul(inputs, self.w) + self.build

  def get_config(self):
    return {'units': self.units}

layer = Linear(units=64)
config = layer.get_config()
print(config)

new_layer = Linear.from_config(config)

The `__init__` method of the base Layer class takes some keyword arguments, in particular a `name` and a `dtype`. It is best practuce to pass these **keyword arguments** to the parent class in `__init__` and include them in the layer config.

In [0]:
class Linear(layers.Layer):

  def __init__(self, units=32, **kwargs):
    super(Linear, self).__init__(**kwargs)
    self.units = units

  def build(self, input_shape):
    self.w = self.add_weight(shape=(input_shape[-1], self.units), initializer='random_normal', trainable=True)
    self.b = self.add_weight(shape=(self.units, ), initializer='random_normal', trainable=True)

  def call(self, inputs):
    return tf.matmul(inputs, self.w) + self.b

  def get_config(self):
    config = super(Linear, self).get_config()
    config.update({'units': self.units})
    return config


layer = Linear(64)
config = layer.get_config()
print(config)
new_layer = Linear.from_config(config)

{'name': 'linear_20', 'trainable': True, 'dtype': 'float32', 'units': 64}


## Model class
To summarize the Layer class:
* A Layer encapsulates a state (in the `build` method) and some computation (in the `call` method)
* Layers can create (in the `call` method) and track losses
* Layers can be recursively nested to create bigger computation blocks


While the Layer class is used to define inner computation blocks, the Model class is used to define the outer model, i.e. the object to be trained. The Model class has the same API as Layer, with the following differences:
* It exposes built-in training (`model.fit`), evaluation (`model.evaluate`), and predicion loops (`model.predict`)
* It exposes the list of its inner layers (`model.layers`)
* It exposes saving (`model.save_weights`) and serialization APIs